In [9]:
import os
import pandas as pd
from consts import *
import autokeras as ak
import tensorflow as tf
from tensorflow.keras.models import load_model

In [2]:
df = pd.read_csv(modified_hotel_reservations_csv_file_path)
df.head(10)


,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,2,0,0,2,0,0,3,23,2018,10,28,4,0,0,0,145.00,2,1
1,3,0,0,1,0,0,0,8,2018,11,5,3,0,0,0,160.00,0,1
2,2,0,2,4,0,0,3,10,2017,9,5,4,0,0,0,110.08,1,1
3,2,0,0,2,3,0,0,51,2018,10,5,4,0,0,0,108.00,1,1
4,1,0,0,1,0,0,0,8,2018,2,19,2,0,0,0,81.00,0,1
5,2,2,1,0,0,0,5,54,2018,7,10,4,0,0,0,156.93,2,1
6,1,0,1,2,0,0,0,32,2018,2,5,4,0,0,0,72.90,0,0
7,2,0,1,4,3,0,0,69,2018,10,31,4,0,0,0,79.38,2,0
8,2,0,1,3,1,0,3,136,2018,12,26,4,0,0,0,138.76,1,0
9,2,0,2,2,0,0,0,104,2018,7,3,4,0,0,0,114.30,1,1


In [3]:
df.columns = df.columns.str.strip()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   no_of_adults                          36275 non-null  int64  
 1   no_of_children                        36275 non-null  int64  
 2   no_of_weekend_nights                  36275 non-null  int64  
 3   no_of_week_nights                     36275 non-null  int64  
 4   type_of_meal_plan                     36275 non-null  int64  
 5   required_car_parking_space            36275 non-null  int64  
 6   room_type_reserved                    36275 non-null  int64  
 7   lead_time                             36275 non-null  int64  
 8   arrival_year                          36275 non-null  int64  
 9   arrival_month                         36275 non-null  int64  
 10  arrival_date                          36275 non-null  int64  
 11  market_segment_

In [4]:
train_size = int(df.shape[0] * 0.90)
df[:train_size].to_csv(hotel_reservations_train_file_path, index=False)
df[train_size:].to_csv(hotel_reservations_test_file_path, index=False)

In [5]:
column_types = {
    "no_of_adults": "numerical",
    "no_of_children": "numerical",
    "no_of_weekend_nights": "numerical",
    "no_of_week_nights": "numerical",
    "type_of_meal_plan": "categorical",
    "required_car_parking_space": "numerical",
    "room_type_reserved": "categorical",
    "lead_time": "numerical",
    "arrival_year": "numerical",
    "arrival_month": "numerical",
    "arrival_date": "numerical",
    "market_segment_type": "categorical",
    "repeated_guest": "numerical",
    "no_of_previous_cancellations": "numerical",
    "no_of_previous_bookings_not_canceled": "numerical",
    "avg_price_per_room": "numerical",
    "no_of_special_requests": "numerical",
}


In [6]:
clf = ak.StructuredDataClassifier(
    column_types=column_types,
    overwrite=True, 
    max_trials=5
)

clf.fit(
    hotel_reservations_train_file_path,
    "booking_status",
    epochs=100,
)

Trial 5 Complete [00h 00m 36s]
val_accuracy: 0.8768261075019836

Best val_accuracy So Far: 0.8768261075019836
Total elapsed time: 00h 01m 49s
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
1021/1021 [==============================] - 1s 613us/step - loss: 0.4192 - accuracy: 0.8125
Epoch 2/100
1021/1021 [==============================] - 1s 616us/step - loss: 0.3850 - accuracy: 0.8288
Epoch 3/100
1021/1021 [==============================] - 1s 601us/step - loss: 0.3707 - accuracy: 0.8354
Epoch 4/100
1021/1021 [==============================] - 1s 598us/step - loss: 0.3595 - accuracy: 0.8407
Epoch 5/100
1021/1021 [==============================] - 1s 598us/step - loss: 0.3503 - accuracy: 0.8429
Epoch 6/100
1021/1021 [==============================] - 1s 615us/step - loss: 0.3443 - accuracy: 0.8473
Epoch 7/100
1021/1021 [==============================] - 1s 600us/step - loss: 0.3394 - accuracy: 0.8503
Epoch 8/100
1021/1021 [==============================] - 1s 601us/step - loss: 0.3342

INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


In [7]:
predicted_y = clf.predict(hotel_reservations_test_file_path)
print(clf.evaluate(hotel_reservations_test_file_path, "booking_status"))

114/114 [==============================] - 0s 457us/step - loss: 0.2855 - accuracy: 0.8740
[0.2855427861213684, 0.8740352988243103]


In [10]:
model = clf.export_model()

try:
    model.save(classification_model_path_name, save_format="tf")
except Exception:
    model.save(classification_model_path_name + ".h5")

os.system("rm -rf ./structured_data_classifier")

INFO:tensorflow:Assets written to: ./trained_models/structured_data_classifier/assets


INFO:tensorflow:Assets written to: ./trained_models/structured_data_classifier/assets


0

In [11]:
loaded_model = load_model(classification_model_path_name, custom_objects=ak.CUSTOM_OBJECTS)
x_test = pd.read_csv(hotel_reservations_predict_file_path)

predicted_y = loaded_model.predict(x_test)
print(predicted_y)

1/1 [==============================] - 0s 61ms/step
[[1.0000000e+00]
 [9.9992871e-01]
 [4.0534758e-14]
 [9.9999988e-01]
 [7.0969220e-03]]
